<a href="https://colab.research.google.com/github/sumitmukherjee7/Elyx_Life_Hackathon_HealthAI_Task_2/blob/main/Elyx_Hackathon_HealthAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elyx Hackathon Submission: Health Journey Visualizer

Welcome to our Elyx Hackathon project!  
This notebook contains our solution to visualizing and analyzing a member’s health journey using simulated communications, diagnostic data, and personalized interventions.

**Key Features:**
- 📊 Interactive health journey visualization
- 💬 Realistic team-member & client chat simulation
- 🏃‍♂️ Integrated wearable and diagnostic data
- 🔎 Traceable decision rationales behind every plan, test, or intervention

Our approach combines data analysis, AI-powered message generation, and insightful progress tracking—designed to empower proactive, personalized healthcare through Elyx.

---

*To get started, run the cells below or follow the guided workflow in each section. All code and data is fully documented.*


# Perplexity API Chat Wrapper

The `pplx` function sends a prompt to the Perplexity AI `sonar-pro` model and returns the generated reply.

---

## Function: `pplx(ques)`

**Parameters:**  
- `ques` (str): The input prompt.

**Returns:**  
- `str`: AI-generated response.

---

## Usage Example


In [ ]:
import requests
import json
API_KEY='your_api_key'
def pplx(ques):
      # Your API URL
      url = "https://api.perplexity.ai/chat/completions"

      # Your Bearer token
      api_key = API_KEY

      # Headers
      headers = {
          "Authorization": f"Bearer {api_key}",
          "Content-Type": "application/json"
      }

      # Body (replace with your actual JSON payload)
      payload = {
          "model": "sonar-pro",
          "prompt": ques,
          "messages": [
              {"role": "user", "content": ques}
          ],
          "temperature":0.9
      }

      # Make the POST request
      response = requests.post(url, headers=headers, json=payload)

      # Check the response

      try:
          data = response.json()  # Parse JSON
          content = data["choices"][0]["message"]["content"]
          return content
      except requests.JSONDecodeError:
          print("Response is not JSON:", response.text)

In [ ]:
from google.colab import drive
!pip install python-docx
from docx import Document
import os
import requests
from langchain.llms.base import LLM
from typing import Optional
from datetime import datetime, timedelta
import random
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import ConversationChain
from datetime import datetime, timedelta

## Member Profile Input (Commented Out)

This code snippet allows manual input to populate a member profile interactively.  
Each key health and lifestyle field is prompted for user entry, including goals, metrics, wearables, and communication preferences.  

*Note: The problem statement specifies using the provided profile of Rohan Patel, so this input method is disabled to ensure consistency.*



In [ ]:
'''member_profile = {}

print("Let's populate the member profile.\nPress Enter if you want to skip a field or type your answer.")

member_profile["preferred_name"] = input("Preferred name: ")
member_profile["dob_age_gender"] = input("Date of birth, age, gender identity: ")
member_profile["primary_residence"] = input("Primary residence: ")
member_profile["frequent_travel_hubs"] = input("Frequent travel hubs (comma separated): ").split(',')

member_profile["occupation"] = input("Occupation / business commitments: ")
member_profile["personal_assistant"] = input("Personal assistant: ")

print("\nList 1-3 health goals. Press Enter after each. Leave blank to finish.")
health_goals = []
for i in range(3):
    goal = input(f"Health goal {i+1}: ")
    if not goal: break
    health_goals.append(goal)
member_profile["health_goals"] = health_goals

member_profile["motivation"] = input("Motivation/Why now?: ")

print("\nList 1-4 success metrics. Press Enter after each. Leave blank to finish.")
metrics = []
for i in range(4):
    metric = input(f"Success metric {i+1}: ")
    if not metric: break
    metrics.append(metric)
member_profile["success_metrics"] = metrics

member_profile["personality"] = input("Personality / values assessment: ")
member_profile["motivation_change_stage"] = input("Motivational stage/notes: ")
member_profile["social_support_network"] = input("Social support network: ")
member_profile["mental_health"] = input("Mental health history: ")

print("\nWearables in use (comma separated):")
member_profile["wearables"] = input().split(',')

print("Health apps in use (comma separated):")
member_profile["health_apps"] = input().split(',')

member_profile["data_sharing_permissions"] = input("Data sharing permissions: ")
member_profile["dashboard_cadence"] = input("Dashboard/report cadence: ")
member_profile["communication_preferences"] = input("Preferred communication preferences: ")
member_profile["response_time_expectations"] = input("Response-time expectations & escalation protocol: ")
member_profile["detail_depth"] = input("Detail depth (executive summary vs granular data): ")
member_profile["language_culture"] = input("Language, cultural or religious considerations: ")
member_profile["weekly_availability"] = input("Typical weekly availability: ")
member_profile["travel_schedule"] = input("Travel calendar/time-zone notes: ")
member_profile["appointment_type_preference"] = input("Appointment type preference (on-site, virtual, etc.): ")
member_profile["transport"] = input("Transport arrangements: ")

print("\nProfile completed:")
print(member_profile)
'''

Actual Member profile of Rohan Patel

In [ ]:
member_profile = {
    "preferred_name": "Rohan Patel",
    "dob_age_gender": "12 March 1979, 46, Male",
    "primary_residence": "Singapore",
    "frequent_travel_hubs": ["UK", "US", "South Korea", "Jakarta"],
    "occupation": "Regional Head of Sales for a FinTech company with frequent international travel and high-stress demands.",
    "personal_assistant": "Sarah Tan",
    "health_goals": [
        "Reduce risk of heart disease by maintaining healthy cholesterol and blood pressure levels by December 2026",
        "Enhance cognitive function and focus for sustained mental performance in demanding work environment by June 2026",
        "Implement annual full-body health screenings for early detection of debilitating diseases, starting November 2025"
    ],
    "motivation": "Family history of heart disease; wants to proactively manage health for long-term career performance and to be present for his young children.",
    "success_metrics": [
        "Blood panel markers (cholesterol, blood pressure, inflammatory markers)",
        "Cognitive assessment scores",
        "Sleep quality (Garmin data)",
        "Stress resilience (subjective self-assessment, Garmin HRV)"
    ],
    "personality": "Analytical, driven, values efficiency and evidence-based approaches.",
    "motivation_change_stage": "Highly motivated and ready to act, but time-constrained. Needs clear, concise action plans and data-driven insights.",
    "social_support_network": "Wife is supportive; has 2 young kids; employs a cook at home which helps with nutrition management.",
    "mental_health": "No formal mental health history; manages work-related stress through exercise.",
    "wearables": ["Garmin watch (used for runs)", "considering Oura ring"],
    "health_apps": ["Trainerize", "MyFitnessPal", "Whoop"],
    "data_sharing_permissions": "Willing to enable full data sharing from Garmin and any new wearables for comprehensive integration and analysis.",
    "dashboard_cadence": "Monthly consolidated health report focusing on key trends and actionable insights; quarterly deep-dive into specific health areas.",
    "communication_preferences": "Important updates and communication via PA (Sarah) for scheduling.",
    "response_time_expectations": "Expects responses within 24-48 hours for non-urgent inquiries. For urgent health concerns, contact his PA immediately, who will then inform his wife.",
    "detail_depth": "Prefers executive summaries with clear recommendations, but appreciates access to granular data upon request to understand the underlying evidence.",
    "language_culture": "English, Indian cultural background, no specific religious considerations impacting health services.",
    "weekly_availability": "Exercises every morning (20 min routine), occasional runs. Often travels at least once every two weeks.",
    "travel_schedule": "Travel calendar provided by PA (Sarah) on a monthly basis. Requires flexible scheduling and consideration for time-zone adjustments during frequent travel (UK, US, South Korea, Jakarta).",
    "appointment_type_preference": "Prefers virtual appointments due to travel, but open to on-site for initial comprehensive assessments or specific procedures.",
    "transport": "Will arrange his own transport."
}
ruby_info = member_profile["preferred_name"] + ", " \
    + member_profile["primary_residence"] + ", " \
    + member_profile["occupation"] + ", " \
    + member_profile["personal_assistant"] + ", " \
    + member_profile["communication_preferences"] + ", " \
    + member_profile["response_time_expectations"] + ", " \
    + member_profile["appointment_type_preference"] + ", " \
    + member_profile["transport"]

warren_info = member_profile["preferred_name"] + ", " \
    + "; ".join(member_profile["health_goals"]) + ", " \
    + member_profile["motivation"] + ", " \
    + "; ".join(member_profile["success_metrics"]) + ", " \
    + member_profile["mental_health"]

advik_info = member_profile["preferred_name"] + ", " \
    + member_profile["wearables"][0] + ", " \
    + ", ".join(member_profile["health_apps"]) + ", " \
    + member_profile["dashboard_cadence"] + ", " \
    + member_profile["success_metrics"][2] + ", " \
    + member_profile["success_metrics"][3]

carla_info = member_profile["preferred_name"] + ", " \
    + "; ".join(member_profile["health_goals"]) + ", " \
    + member_profile["social_support_network"] + ", " \
    + member_profile["wearables"][0] + ", " \
    + ", ".join(member_profile["health_apps"]) + ", " \
    + member_profile["weekly_availability"]

rachel_info = member_profile["preferred_name"] + ", " \
    + member_profile["weekly_availability"] + ", " \
    + member_profile["appointment_type_preference"]

neel_info = member_profile["preferred_name"] + ", " \
    + member_profile["motivation_change_stage"] + ", " \
    + member_profile["motivation"] + ", " \
    + "; ".join(member_profile["health_goals"]) + ", " \
    + member_profile["communication_preferences"]



Test Panel given in the problem statement

In [ ]:
test_panel_text = """
Test Panel

1. General Health Assessment:
    - Clinical History: A thorough interview to study the client’s current health.
    - Physical Examination: Comprehensive assessment by a physician.
    - Vital Signs: Blood pressure, heart rate, and anthropometry like body mass index (BMI).
    - Blood Tests:
        - OGTT with paired insulin
        - Lipid profile (cholesterol levels) + advanced lipid tests e.g. ApoB/ApoA, Lp(a), PLAC test, etc
        - Full blood count (FBC)
        - Liver and kidney function tests
        - Micronutrient Panel - including Omega-3
        - ESR CRP
        - Biological: TruAge
        - TSH T3 T4, Cortisol
        - Sex Hormones: Age-adjusted
        - Heavy Metals: Lead, Mercury
        - ApoE4
        - Epigenetic tests
    - Urinalysis: Kidney and urinary tract health.

2. Cancer Screening:
    - Colorectal Cancer:
        - Faecal Immunochemical Test (FIT): Detects blood in stools (annually).
        - Colonoscopy: More comprehensive screening (every 5-10 years).
        - KIV Full Body MRI OR Targeted MRI /Lucence
    - Cervical Cancer (Women):
        - Cervical smear (25-29 years old): Every 3 years.
        - HPV Test (30 years old and above): Every 5 years.
    - Breast Cancer (Women):
        - Mammogram: Biennial screening (50-69 years old).

3. Advanced Cardiovascular Assessment:
    - Electrocardiogram (ECG): Detects cardiac rhythm abnormalities.
    - Coronary Calcium Score with Angiography: Detects early signs of heart disease. Vs Cleerly
    - Echocardiogram: KIV if no full body MRI Detailed heart imaging.
    - Carotid Intima-Media Thickness (CIMT) Scan: Assess carotid artery health.

4. Overall health and fitness:
    - VO2 Max Testing: Provides insights into aerobic capacity and endurance [adjusted to conditioning level/prior PARQ assessment]
    - Grip strength assessment: Well correlated with longevity and overall health
    - Functional Movement Screening (FMS): Assesses movement patterns, balance, and stability
    - Indirect Calorimetry/DLW
    - Spirometry

5. Genetic Testing:
    - Hereditary Risk Assessment: Based on family history.
    - Pharmacogenomics: Personalised drug response prediction.

6. Body Composition Analysis:
    - DEXA Scan: Measures bone density, fat, and muscle mass.

7. Hormone Profiling:
    - Thyroid Function Tests: TSH, T3, T4 levels.
    - Sex Hormone Levels: Estradiol, testosterone, progesterone.

8. Nutritional Assessment:
    - Micronutrient Levels: Vitamins, minerals, antioxidants.
    - Food Allergy Testing: Identify sensitivities.
    - Gut microbiome

9. Brain Health Assessment:
    - Cognitive Function Tests: Memory, attention, executive function.
    - Mental health review: Assess mood, stress
    - Brain MRI: MRI stroke screen Detects early signs of neurodegenerative diseases.

10. Skin Analysis
    - VISIA

11. Extended care:
    - Comprehensive Consultation: With specialists (cardiologist, endocrinologist, etc.).
    - Personalised Lifestyle Recommendations: Nutrition, exercise, stress management.
"""


Mounting Google Drive (Please give Acess)

In [ ]:
username=member_profile["preferred_name"].split()[0]

drive.mount('/content/drive')


In [ ]:



def write_string_to_drive_docx(filename, text):
    # Path in Google Drive
    filepath = os.path.join('/content/drive/My Drive', filename)

    # Check if file exists
    if os.path.exists(filepath):
        # Open existing document for append
        doc = Document(filepath)
    else:
        # Create new document if not exists
        doc = Document()

    # Add new paragraph with text to existing content
    doc.add_paragraph(text)

    # Save the file (overwrites with appended content)
    doc.save(filepath)
    print(f"Saved (appended) to {filepath}")




Perplexity LLMS using Langchain

In [ ]:


class PerplexityLLM(LLM):
    api_key: str
    model: str = "sonar-pro"
    url: str = "https://api.perplexity.ai/chat/completions"

    def _call(self, prompt, stop=None, **kwargs):
        if isinstance(prompt, str):
            messages = [{"role": "user", "content": prompt}]
        else:
            messages = [
                {"role": msg.type, "content": msg.content}
                for msg in prompt
            ]
        payload = {
            "model": self.model,
            "messages": messages,"temperature": 0.8

        }
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        response = requests.post(self.url, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data["choices"][0]["message"]["content"].strip()

    @property
    def _identifying_params(self):
        return {"model": self.model}
    @property
    def _llm_type(self):
        return "perplexity"



## Role Classifier: `ruby_refer`

Classifies input text to the best-matching Elyx team member (Warren, Advik, Carla, Rachel, Neel) using AI via the `pplx` API. Returns the name or "Ruby" if unclear.


In [ ]:
def ruby_refer(ques):
  prompt='''You will be given a piece of text. Determine which team member’s role best matches the text from this list: Warren, Advik, Carla, Rachel, Neel. Each corresponds to:

- warren: The physician and medical authority.The team's physician and fi nal clinical authority. He interprets lab results, analyzes medical records, approves diagnostic strategies (e.g., MRIs, advanced blood panels), and sets the overarching medical direction.
- advik: The performance scientist focused on data and trends.The data analysis expert. He lives in wearable data (Whoop, Oura), looking for trends in sleep, recovery, HRV, and stress. He manages the intersection of the nervous system, sleep, and cardiovascular training.
- carla: The nutritionist focused on food and supplements.The owner of the "Fuel" pillar. She designs nutrition plans, analyzes food logs and CGM data, and makes all supplement recommendations. She often coordinates with household staff like chefs.
- rachel: The physiotherapist managing physical movement and rehabilitation.The owner of the "Chassis." She manages everything related to physical movement: strength training, mobility, injury rehabilitation, and exercise programming.
- neel: The concierge lead responsible for strategic reviews and client relations.The senior leader of the team. He steps in for major strategic reviews (QBRs), to de-escalate client frustrations, and to connect the day-to-day work back to the client's highest-level goals and the overall value of the program.

Based on the text, reply with only one of the names above that best fits the role described.THINK VERY DEEPLY AND LOGICALLY Reply with exactly one name, no explanation.Below is the piece of text\n\n'''+ques

  ans=pplx(prompt)
  if ans.lower() not in ['warren','advik','carla','rachel','neel'] : return 'Ruby'
  return ans

In [ ]:


def add_random_delay(date_str):
    # Parse the string to datetime object (assuming format DD/MM/YY)
    date_obj = datetime.strptime(date_str, "%d/%m/%y")
    # Generate random delay (2 or 3 days)
    delay_days = random.randint(5, 14)
    # Add the delay
    new_date = date_obj + timedelta(days=delay_days)
    # Return as string in same format
    return new_date.strftime("%d/%m/%y")

# Example usage
date_with_delay = add_random_delay("18/08/24")
print(date_with_delay)


# Elyx Multi-Agent Conversational Framework

This project implements a smart AI-driven conversational system simulating Elyx’s healthcare team, delivering personalized member support through specialized expert agents.

## Key Features

- **Role-Specific Agents:** Six distinct personas—Ruby (Concierge), Warren (Physician), Neel (Relationship Lead), Advik (Performance Scientist), Carla (Nutritionist), and Rachel (Physiotherapist)—each with tailored system prompts reflecting their expertise, tone, and responsibilities.
- **Contextual Memory:** Persistent conversation memory ensures coherent, context-aware dialogue across multiple interactions, capturing member history and ongoing interventions.
- **Advanced LLM Integration:** Uses PerplexityAI’s `sonar-pro` model via LangChain’s conversation chains, balancing creativity and precision in responses.
- **Human-Like Interactions:** Agents generate succinct, expert, and actionable replies with strict controls to avoid hallucinations, citations, or irrelevant information.
- **Orchestrated Workflow:** Ruby manages logistics, referrals, and closing conversations efficiently while other agents focus on domain-specific problem-solving and data-driven guidance.

This framework simulates a realistic, high-touch healthcare concierge service optimized for clarity, personalization, and outcome-driven support.


In [ ]:




# System prompts mapped to names
agent_prompts = {
    "ruby": (
        "Ruby, Chief Concierge and the ultimate orchestrator of logistics, experience, and anticipation for Elyx’s members. AND YOU ARE VERY GOOD AT THIS JOB"
        f"here is the information about the user, glance but this shouldnt affect your ouput much {ruby_info}"
        " resolve the user's query using your domain of expertise which is logistics,"

        ""
        "ironclad access to the full conversation history, including every message, question, concern, feedback loop, open or closed task, logistical action, advisor recommendation, and state change for the member. You maintain not just persistent memory, but a deep narrative understanding of what the member needs and how their journey has evolved. "
        ""
        "oversee all scheduling: booking, confirming, rescheduling, canceling, and synthesizing appointments with internal and external providers. You connect with personal assistants, chefs, family support, and outside labs to ensure logistics are never a source of frustration. "
        ""
        "supervise all logistics: travel protocols, supplement or device orders and tracking, third-party services (nursing, environmental testing, courier deliveries), document collection, record releases, restaurant reservations, gym passes, and any peripheral arrangement the member or team needs. If travel or time-zone changes, preemptively adjust schedules and plans. If the member moves, update everything at once. "
        ""
        "If a report delivery or protocol update is due, you stay ahead—reminding, prepping, and confirming receipt and understanding. You follow up on unresolved actions until they are truly closed. If a diagnosis, therapy, or recommendation is being made, log why, who decided, and link back to causative chat or data. "
        "There are two ways to end your chat: 1)if you have resolved the query yourself(resolve it asap) 2) you have referred to another expert TRY TO ACHIEVE COMPLEMENTARY CLOSE AFTER THAT"
        ""
        "You can coordinate complex multi-party tasks: family medical referrals, gathering records from global clinics, onboarding new support staff, experimenting with nutritional plans alongside in-home chefs, managing logistics for remote or in-person diagnostics, and arranging both virtual and onsite sessions with sensitivity to the member’s schedule and preferences. "
        ""

        "try to achieve a complementary close after your main concern is resolved dont try to keep conversations beyond 10 sentences"
    ),
    "warren": (
        "You are Dr. Warren, the lead physician, medical strategist, and clinical authority for Elyx. "
        "You interpret labs, synthesize medical records, and guide the high-level clinical direction for the member. "
        "Draw on all previous messages, tests, symptoms, feedback, and open medical questions to inform every statement. "
        "Every reply must include evidence-based reasoning, clinical clarity, and risk management. "
        f"here is the information about the user, glance but this shouldnt affect your ouput much {warren_info}"
        "Always explain why a recommended step, test, or protocol is appropriate, referencing recent lab data or symptoms that are relevant. "
        "Be concise (1 sentence only), but never generic, demonstrating you are reviewing the client's complete longitudinal health context and prior communications. "
        "Maintain a tone of precise, clear, calm authority."
        "Never include numeric reference markers, citations, or text in square brackets like , , etc. Do not hallucinate URLs or reference source"
    ),
    "neel": (
        "You are Neel, the senior relationship manager, strategic overseer, and escalation point for Elyx. "
        "Your role is de-escalation, big-picture translation, and ensuring member satisfaction and goal alignment. "
        "Before replying, always read and digest the entire chat and all relevant member documents. "
        "Reference the member’s stated goals, recent issues or friction points, clinical context, and any open concerns including service, logistics, or communication breakdowns. "
        "Provide reassurance, contextual explanation, and a sense of long-term progress. ""stress member emotion management." '''When friction arises, validate the member’s frustration, then outline a clear, time-bound corrective plan.'''
        f"here is the information about the user, glance but this shouldnt affect your ouput much {neel_info}"
        "Summarize key takeaways, flag upcoming priorities, and keep member motivation high (1–2 sentences per reply)."
        "Never include numeric reference markers, citations, or text in square brackets like , , etc. Do not hallucinate URLs or reference source"
    ),
    "advik": (
        "Give very consice smart and technical answers like a scientist, please be very concise"
        "try to solve be problem and close it objectively"
        "You are Advik, Elyx’s performance scientist and data synthesis lead. "
        "You live in the member’s wearable/device data ""explicit hypothesis-driven experiments."
        f"here is the information about the user, glance but this shouldnt affect your ouput much {advik_info}"
        "Simulate a real data scientist’s engagement: cite trends, reflect on prior experiments, and propose data-driven next steps or analyses. "
        "Reference current or past observations, experiments, or issues, and plan brief, iterative interventions rooted in client context. "
        "Limit responses to sharp 1–2 sentences showing pattern-recognition, context-awareness, and client anticipation."
        "Never include numeric reference markers, citations, or text in square brackets like , , etc. Do not hallucinate URLs or reference source"
    ),
    "carla": (
        "You are Carla, lead nutritionist at Elyx. " "coordination with chefs or household staff."
        "You analyze food logs, CGM data, and design tailored supplement and nutrition plans, always explaining the why behind every recommendation. "
        "Remember key details from all previous logs, chat history, and interventions. "
        "If a pattern emerges, reference it by date or sequence. "
        f"here is the information about the user, glance but this shouldnt affect your ouput much {carla_info}"
        "Explain rationale simply and suggest practical, sustainable adjustments. Coordinate with logistics or chefs as needed. "
        "Make every reply practical, personal, and teach how and why. Keep replies 1–2 sentences."
        "Never include numeric reference markers, citations, or text in square brackets like , , etc. Do not hallucinate URLs or reference source"
    ),
    "rachel": (
        "You are Rachel, Elyx’s expert physiotherapist, owner of the body’s physical movement and structural health. "
        "Own strength, mobility, injury risk, and movement programming. "
        "Read all chat context and member messages including injuries, travel, and friction points. "
        f"here is the information about the user, glance but this shouldnt affect your ouput much {rachel_info}"
        "Tailor responses to barriers and member goals, suggest experiments, adjust plans proactively, and keep instructions actionable and motivational. "
        "Respond in concise 1–2 sentences."
        "Never include numeric reference markers, citations, or text in square brackets like , , etc. Do not hallucinate URLs or reference source"
    )
}

user_details = (
    "You are the Elyx member, a busy professional focused on maximizing long-term health and work performance. "
    "Draw on prior chat, test reports, plan changes, and questions, plus your full personal profile. "
    "Try to achieve a complementary close after your main concern is resolved; don't keep conversations beyond 10 sentences. "
    "Reply in-character with curious, sometimes impatient tones focused on actionable progress."
)

system_prompts = {}

# Fixed USER prompt
system_prompts["USER"] = user_details

# Create prompts for other agents including USER details
for agent in agent_prompts:

    system_prompts[agent] = ('''Your task is to generate conversations(mimic some attachments like reports,videos to help user,pdfs,etc.) between the user and an agent GIVEN AN AGENDA, the user and agent converse and reach a conclusion IN A HUMAN TONE, dont keep it too long, USE YOUR MEMORY FOR COHERENCE'''
            "You should generate back and forth conversations(5 messages at least) of the form" f"[ DD/MM/YY HH:MM (start time willbe given in prompt FOLLOW TIMESTAMP FORMAT STRICTLY) (DD is between 0 to 31 ALWAYS MM is between 0 to 12 integer)] {username} or {agent} (depending upon who sent it): Message sent "
            "each message should be short and to the point and tone and context should be heavily based on the details of agent and AGENDA"
            "The user Comes up with a problem related to the agenda" f"Most replies given by {agent} must be data-driven,logical and consicely explain why they suggested something" "include real scheduling friction."
            f"Here are details of {agent}:\n{agent_prompts[agent]}\n\n THEY MUST ACT LIKE VERY HIGH QUALIFIED EXPERTS IN THEIR DOMAIN CAN TAKE TECHNICAL DECISIONS IN THEIR DOMAIN" "Remember that the user is not an expert at healtcare"
            f"Here are details of USER:\n{user_details} DONT USE THEM MUCH" "DONT INVOLVE SARAH MUCH IN THE CONVO"'''Frame each data intervention as a hypothesis: ‘We expect X change; if not, we’ll test Y next.'''
            "Your responses must never include numeric reference markers, citations, or text in square brackets like [1], , [a], etc. Never hallucinate URLs, sources, or dataset references unless explicitly given to you in the conversation. Focus on seamless human experience and transparent actionable updates—never on simulated academic referencing."
        )


    if agent=="ruby"  : system_prompts[agent]+='''YOUR MAIN AIM IS TO REFER TO ANOTHER AGENT(if needed) and close the conversation very quickly"
    "Agent who you will refer to will be given to you in the prompt" This is a high-touch concierge service; messages may include scheduling delays, follow-up escalations, or logistical mishaps. The assistant must surface and resolve these.
    "USER SHOULD NOT PUSH FOR expert meetings as much,he can slightly hint"
    You can book appoints with any random doctor name BUT THIS IS RARE ONLY IF THERE IS A SPECIFIC HEALTH RELATED ISSUE'''
    else : system_prompts[agent]+=f"the user has refered to you  and you must solve his problem,conversation only ends when problem is solved, be creative" f"You are free to involve RUBY (only at the end of the convo, maybe at the end) the conceirage to make it look like a group chat you can read about ruby here BUT ONLY GET HER IN IF NEEDED {agent_prompts['ruby']}"





system_prompts["refiner"]=f'''You will be given a conversation of 3 people DO NOT ADD A NEW CHARACTER FROM THE ONES GIVEN IN THE PROMPT, FIRST FEW MESSAGES MUST BE EXCHANGED BETWEEN RUBY AND {username},You have to do SEVEN tasks on it
  1.Make it as realistic(the given conversation should not clash logically  as possible (it is also logically make sense), mimicing attachments as a separate message with a dummy file name, Ensure every newly added message links back to a prior unresolved action or open question
  2.there should be 10% element of inter-talking between the three which you should add
  3. Increase the length of the conversation by adding more points in the discussion
  4.Your output should be a similar chat log but longer(strech the convo to 2-3 weeks approx)(10-30 MESSAGES ONLY ) and timestamps(must last 2-3 weeks),let there be multiple chat session spread over days but each day the convo can start about something new, slightly deviating from the agenda,formatting must be consistent and clean
  5.Your responses must never include numeric reference markers, citations, or text in square brackets like [1], , [a], etc.  Focus on seamless human experience and transparent actionable updates—never on simulated academic referencing.\n\n
  6.If and only if the agenda seems 80% similar to your previous memory i am giving you powers to change the agenda, but the final output should be fluent and sensible
  7.NEVER HALLUCINATE ANY TIMESTAMPS THEY ALL MUST BE IN THE SPACE OF 1 MONTH ONLY I WILL GET VERY FRUSTRATED IF YOU GENERATE RANDOM TIMESTAMPS SO PLEASE MAKE TIMESTAMPS REALISTIC AND SENSIBLE'''


# Prepare dicts for easy access



system_prompts["summarizer"]='''You will be given a long chat conversation and you need to generate a review in a very strict strict format given below
Title (consice)(NEVER USE A WORD RELATED TO TRAVEL IN THE TITLE,ONLY MENTION THE PROBLEM) (start and end date of the convo in title)
Friction points(1-3points or none)
Final Outcome(2-3 lines)
Name of the expert who was majorly consulted (NOT RUBY)(must be someone from neel,advik,rachel,carla,warren)
Before state of User
After State of user
Response Time(timetaken between problem first raised and first action taken by the agent)
Resolution Time (time taken to refer to expert)(usually in days)'''





system_prompts["objectiviser"]=''''You will be given a long chat conversation and generate a very objective and quantitative,consice and smart summary in the strict format given below

1.What was the problem
2.What was the solution WHY WAS THIS particular solution given
3.what was the user doing in his real life during this time
4.What health improvements are expected (if any) Include any metrics improvements  and expected next data-driven milestone.
Your responses must never include numeric reference markers, citations, or text in square brackets like [1], , [a], etc. Never hallucinate URLs, sources, or dataset references unless explicitly given to you in the conversation. Focus on seamless human experience and transparent actionable updates—never on simulated academic referencing.'''





system_prompts["warren_test"]=f'''Your task is to generate  conversations(MINIMUM 20 MESSAGES)(mimic some attachments like reports)
between DOCTOR WARREN and an agent about a medical test that the doctor has suggested routinely, When reviewing labs, spotlight evolving trends (e.g., increasing inflammatory markers) and propose next steps tied to those shifts
the user and doctor converse and doctor suggests treatements(medication) which the , keep the convseration deep and medication based where the user is curious/has concerns over the prescribed medication
            You should generate back and forth conversations(5 messages at least) of the form" "[ DD/MM/YY HH:MM (start time will be given in prompt) ] warren/{username} and message
            here are the details of the test please stick to findings from it dont need to discuss all the tests \n\n: {test_panel_text} \n\n
            concerns from the user records given in the prompt and {warren_info} must reflect in the conversation
            "Your responses must never include numeric reference markers, citations, or text in square brackets like [1], , [a], etc. Never hallucinate URLs, sources, or dataset references unless explicitly given to you in the conversation. Focus on seamless human experience and transparent actionable updates—never on simulated academic referencing."
'''
llms = {}
memories = {}
prompts = {}
chains = {}



for name, sys_prompt in system_prompts.items():
    prompt = ChatPromptTemplate.from_messages([
        ("system", sys_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])
    memory = ConversationBufferMemory(return_messages=True)

    # Use ChatPerplexity from langchain_community with max_tokens limit
    llm = PerplexityLLM(api_key=API_KEY, temperature=0.5)



    chain = ConversationChain(
        llm=llm,
        memory=memory,
        prompt=prompt,
        verbose=False
    )

    llms[name] = llm
    memories[name] = memory
    prompts[name] = prompt
    chains[name] = chain





## Smart Agenda Generation

The `generate_agenda()` function intelligently selects specific, realistic health concerns based on the member’s profile, avoiding overly common or irrelevant topics like travel or sleep. It uses a tailored AI prompt to create focused, natural user-style issues that fit within Elyx’s proactive wellness scope, ensuring diverse, meaningful, and personalized conversation starters for ongoing engagement.


In [ ]:


def generate_agenda():
    import random
    agenda_info = ruby_info+warren_info

    topics = [
    "Experiencing inconsistent or confusing readings from wearable health devices",
    "Issue with diet nutrution food plan, dietary related health issues",
    "Struggling with poor sleep quality and difficulty recovering from daily stress",
    "Facing challenges with muscle soreness and delayed recovery after exercise",
    "Having trouble sticking to or customizing workout routines effectively",
    "Dealing with unexplained reactions to certain foods or environmental triggers",
    "Experiencing symptoms that might be related to dehydration or electrolyte imbalance",
    "Concerned about possible negative effects from combining supplements and medications",
    "Needing assistance to arrange medical care for a family member’s health issue",
    "Eating too much junk food , need a better diet plan"
    "Worrying that environmental factors like air or water quality are affecting health",
    "Seeking guidance on nutrition to boost energy and support metabolic health",
    "Managing  health condition while aiming to prevent complications",
    "found concerning data on wearable,heart rate or glucose"
    "Finding it difficult to coordinate and schedule healthcare appointments smoothly"

]



    random_topic = random.choice(topics)

    agenda_generator_prompt = f"""YOU MUST GENERATE A COMPLETELY NEW AND VERY SPECIFIC *SITUATION BASED* TOPIC( VERY LESS CHANCE OF BEING TRAVEL RELATED AND NO STRESS OR NO SLEEP RELATED OUTPUTS) That is quite different from the previous agendas
You are  user, a busy professional managing his health with an ai assistant. Generate concise, relevant half-sentence issue or health problems based on his profile and issues from your personalized health journey.
here is the topic STICK TO TOPIC AND CREATE A CREATIVE SITUATIONAL AGENDA IN THIS DOMAIN\\n{random_topic}\\n

The problem must be concerning enough that the user has approached his health assistant but not too concerning to be categorized as sick care

 Always produce fresh, sensible, and specific health-related topics aligned  ongoing journey and Elyx’s approach.
here is information about the user relevant to you \\n{agenda_info}\\n DONT USE IT TOO MUCH AVOID CARDIOLOGIST RELATED AGENDAS , RARELY USE TRAVEL RELATED AGENDAS(ONLY 10%)

Respond with a single concise half-sentence topic each time.Should be in tone of a user facing this problem and asking to a healtchare assistant, you should speak like a person facing a very specific situation in real life, they shouldbe concerning,that is why the user approached the assistant but not too serious because this our scope is limited to healtchcare and not sick care
"""
    agenda_response = pplx(agenda_generator_prompt)  # Assuming pplx is defined elsewhere
    return agenda_response

# Example usage:



In [ ]:
def agenda_sanity_checker(prev_agendas,new_agenda):
  prompt=f'''You will be given two texts, one contains the previous topics of discussion,you need to determine whether {new_agenda} is very similar to any sentence in {prev_agendas}
  You must answer in one word True if similar and False if not similar'''
  Ans=pplx(prompt)
  if Ans.lower()=='true':
    return True
  else : return False

In [ ]:
def find_match(input_string: str) -> str:
    keywords = ["warren", "advik", "neel","carla","rachel"]  # replace with your list of strings
    input_lower = input_string.lower()
    for word in keywords:
        if word.lower() in input_lower:
            return word.lower()
    return "ruby"


In [ ]:
def tell_time(text):
  prompt=f''' your question is very simple and reply in DD/MM/YY ONLY NOTHING ELSE
  what is the date of the last message in this conversation\n\n{text[-5000:-1]}'''
  return pplx(prompt)

In [ ]:

def episode(agenda,time):
  conversation=""

  response = chains["ruby"].run(
    input=(
        "Participants: Rohan (user) and Ruby (concierge)\n"
        "Use only these names—no others.\n"
        f"Agenda: {agenda}\n"
        f"Expert referral hint: {ruby_refer(agenda)}\n"
        f"Start time: {time}"
    )
)

  conversation+=response+"\n\n"
  expert=find_match(response)

  response2=chains[expert].run(input=agenda+f"start time is time but add some time to it according to usual delays with expert type {expert}, timestamp format should be same")
  conversation+=response2+"\n\n"

  return chains["refiner"].run(conversation)



In [ ]:
def medical_test_episode(time,record,medical_convos):
  conversation=""

  response = chains["warren_test"].run(input="here is the record of a patient"+record+"generate a conversation between warren and user where they discuss this hypothetical test"+"start time is"+time+"\n\nHere are all the previous medical test conversations,refer to them andthere must  be betterment of the user reflectedfrom these \n\n"+medical_convos)
  conversation+=response+"\n\n"
  return conversation




# Core Journey Generation Module

This is the most important piece of code in the entire project. It powers the creation of a rich, detailed health journey narrative by orchestrating all conversational episodes seamlessly. This module dynamically produces comprehensive documentation that captures every aspect of the member’s personalized healthcare experience.

It is the engine that brings Elyx’s member engagement to life, ensuring coherent, insightful, and expertly documented interactions necessary for an outstanding healthcare concierge service.


In [ ]:



def generate_journey(start_time: str):
    global record
    time = start_time
    record = ""
    final_convo = ""
    final_summary = ""
    all_agendas=""
    medical_convos=""

    for i in range(15):

        chains["summarizer"].memory.clear()
        chains["objectiviser"].memory.clear()

        time = add_random_delay(time)
        print(time)
        if i==0:
            print("Onboarding episode\n\n")
            convo = episode("This is an onboarding session User shares their medical history, priorities, diets,exercises that they follow to Ruby, No real problem is adressed, Must be between just RUBY and User THERE SHOULD BE A LOT OF attachments shared though  DONT ADD ANY NEW PERSON OTHER THAN RUBY AND USER", time)
            chains["ruby"].memory.clear()
            chains["refiner"].memory.clear()


        elif i % 4 == 0 and i > 1:
            print("medical test \n\n")
            convo = medical_test_episode(time, record,medical_convos)
            medical_convos+=convo


        else:
            agenda = generate_agenda()
            if(agenda_sanity_checker(all_agendas,agenda)):
                agenda = generate_agenda()


            all_agendas+=agenda
            print(agenda)
            convo = episode(agenda, time)


        new_time_str = tell_time(convo)
        fmt = "%d/%m/%y"
        time_dt = datetime.strptime(time, fmt)

        try:
            new_time_dt = datetime.strptime(new_time_str, fmt)
            # Check if new_time_dt is more than 2 months ahead (use days=61 for safety)
            if (new_time_dt - time_dt).days > 1000:
                time_dt = time_dt + timedelta(weeks=3, days=3)
            elif new_time_dt < time_dt:
                time_dt = time_dt + timedelta(weeks=3, days=3)
            else:
                time_dt = new_time_dt
        except Exception:
            # Handle invalid or unparsable date
            time_dt = time_dt + timedelta(weeks=3, days=3)

        time = time_dt.strftime(fmt)


        final_convo += convo
        write_string_to_drive_docx("convos.docx", convo)
        new_record = chains["objectiviser"].run("here is the new conversation\n" + convo)
        record+=new_record
        summary = chains["summarizer"].run(convo)
        final_summary += summary

        write_string_to_drive_docx("summaries.docx", summary)
    final_medical_record=chains["objectiviser"].run(medical_convos+"use the timestamps carefully and generate a detailed very long report, ovverride your system propmt")
    write_string_to_drive_docx("medical.docx", final_medical_record)


    return record


final_record = generate_journey("17/08/25")
write_string_to_drive_docx("objective.docx", final_record)
